In [1]:
%load_ext line_profiler

from pathlib import Path
import sys

import numpy as np

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages']


# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [2]:
import pandas as pd
import src.pipeline.rag as rag
import glob 
import os
import time

# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Loading saved comparator sets

In [3]:
ms_data = pd.read_parquet('output/comparator-sets/maintained_schools.parquet')
ms_comparators = pd.read_parquet('output/comparator-sets/ms_comparators.parquet')

Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [4]:
import warnings

def compute_rag(data, comparators):
        # TODO: This shouldn't be required
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        with np.errstate(invalid="ignore"):
            indices = range(10)
            keys = rag.category_settings.keys()
            st = time.time()
            for indx in indices:
                target = data.iloc[indx]
                urn = target.name
                pupil_urns = comparators["Pupil"][urn]
                building_urns = comparators["Building"][urn]
                for cat_name in keys:
                    rag_settings = rag.category_settings[cat_name]
                    set_urns = pupil_urns if rag_settings["type"] == "Pupil" else building_urns
                    if set_urns is not None:
                        comparator_set = ms_data[ms_data.index.isin(set_urns)]
                        for r in rag.compute_category_rag(cat_name, rag_settings, target, comparator_set):
                            yield r 
                if indx > 1 and indx % 100 == 0:
                   print(f'Completed {indx} RAGs in {time.time() - st:.2f} secs')
                   st = time.time()

In [23]:
r = compute_rag(ms_data, ms_comparators)

pd.DataFrame(r)

Completed 100 RAGs in 4.59 secs
Completed 200 RAGs in 9.09 secs
Completed 300 RAGs in 13.46 secs
Completed 400 RAGs in 18.22 secs
Completed 500 RAGs in 22.77 secs
Completed 600 RAGs in 27.33 secs
Completed 700 RAGs in 31.71 secs
Completed 800 RAGs in 36.22 secs
Completed 900 RAGs in 40.74 secs


,Urn,Category,Sub category,Value,Mean,Diff Mean,Key,Percentage Diff,Percentile,Decile,Rag
0,100000,Teaching and Teaching support staff,Teaching staff,315.217183,386.088780,-70.871597,outstanding,-22.483418,17,1,amber
1,100000,Teaching and Teaching support staff,Supply teaching staff,3.386822,1.275625,2.111197,outstanding,62.335632,24,2,amber
2,100000,Teaching and Teaching support staff,Education support staff,174.010819,164.340138,9.670681,outstanding,5.557517,58,5,green
3,100000,Teaching and Teaching support staff,Agency supply teaching staff,19.892321,14.264311,5.628010,outstanding,28.292373,45,4,green
4,100000,Teaching and Teaching support staff,Educational consultancy,13.436894,10.490978,2.945916,outstanding,21.924085,59,5,green
...,...,...,...,...,...,...,...,...,...,...,...
40432,102308,Other costs,Special facilities,18.714781,1.861483,16.853298,other,90.053406,62,6,amber
40433,102308,Other costs,PFI charges,0.000000,0.000000,0.000000,other,0.000000,0,0,green
40434,102308,Other costs,Interest charges for loan and bank,0.000000,0.000000,0.000000,other,0.000000,0,0,green
40435,102308,Other costs,Direct revenue financing,9.204200,0.000000,9.204200,other,100.000000,17,1,green


In [6]:
%lprun -f rag.get_category_series list(compute_rag(ms_data, ms_comparators))